In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm import tqdm

# ============================================================
# 🔹 ENABLE TQDM PROGRESS BARS FOR PANDAS
# ============================================================
tqdm.pandas()  # adds progress_apply to pandas


# ============================================================
# 1️) LOAD DATA (with progress tracking & efficient cleaning)
# ============================================================

# Initialize tqdm progress bar for chunks
chunksize = 500_000
filename = "All Recorded Traffic.txt"

# Count total lines first (optional, but lets tqdm show % complete)
with open(filename, 'r') as f:
    total_lines = sum(1 for _ in f)
total_chunks = total_lines // chunksize + 1

parts = []

# Read in chunks with progress bar
for ch in tqdm(pd.read_csv(filename, sep="\t", dtype=str, chunksize=chunksize),
               total=total_chunks, desc="Reading and cleaning chunks"):
    # Clean column names
    ch.columns = ch.columns.str.strip()

    # Strip whitespace only in string columns
    str_cols = ch.select_dtypes(include=["object"]).columns
    ch[str_cols] = ch[str_cols].apply(lambda col: col.str.strip())

    parts.append(ch)

# Combine all chunks into one dataframe
traffic = pd.concat(parts, ignore_index=True)

print(f"\n Finished loading. Shape: {traffic.shape}")


Reading and cleaning chunks: 100%|██████████████| 11/11 [00:19<00:00,  1.80s/it]



 Finished loading. Shape: (5383378, 30)


In [5]:
# ============================================================
# 2️) NORMALIZE DATE / TIME
# ============================================================

traffic['DATE'] = pd.to_datetime(traffic['DATE'], errors='coerce')

def normalize_time(t):
    """Standardize various time formats into consistent HH:MM:SS."""
    if pd.isna(t):
        return pd.NaT
    s = str(t).strip()
    if ':' in s:
        s = s.split('.')[0]
        for fmt in ['%H:%M:%S', '%H:%M']:
            try:
                return pd.to_datetime(s, format=fmt).time()
            except:
                continue
        return pd.NaT
    digits = ''.join(ch for ch in s if ch.isdigit()).zfill(4)[-4:]
    try:
        return datetime.strptime(digits, '%H%M').time()
    except:
        return pd.NaT

print("\n Normalizing TIME column...")
traffic['TIME_parsed'] = traffic['TIME'].progress_apply(normalize_time)

traffic['DATETIME'] = traffic.progress_apply(
    lambda r: pd.NaT if pd.isna(r['DATE']) or pd.isna(r['TIME_parsed'])
    else pd.to_datetime(f"{r['DATE'].date()} {r['TIME_parsed'].strftime('%H:%M:%S')}"),
    axis=1
)


 Normalizing TIME column...


100%|███████████████████████████████| 5383378/5383378 [12:26<00:00, 7208.84it/s]


In [7]:
# ============================================================
# 3️) NUMERIC CONVERSION
# ============================================================

num_cols = ['TOTAL', 'CASH', 'EZPASS', 'VIOLATION', 'Autos', 'Small_T', 'Large_T', 'Buses']

print("\n Converting numeric columns...")
for c in tqdm(num_cols, desc="Numeric conversion"):
    if c in traffic.columns:
        traffic[c] = pd.to_numeric(traffic[c].replace(['NULL', ''], np.nan), errors='coerce')
print("Numeric columns converted")

# --- Convert CLASS columns to numeric to avoid Parquet errors. And Fill nulls in CLASS columns with 0
class_cols = [c for c in traffic.columns if c.upper().startswith('CLASS')]
print("\nConverting CLASS columns and filling NaNs with 0...")
for c in tqdm(class_cols, desc="CLASS numeric conversion"):
    traffic[c] = pd.to_numeric(traffic[c], errors='coerce').fillna(0).astype(float)
print("CLASS columns converted and NaNs filled")


 Converting numeric columns...


Numeric conversion: 100%|█████████████████████████| 8/8 [00:11<00:00,  1.49s/it]


Numeric columns converted

Converting CLASS columns and filling NaNs with 0...


CLASS numeric conversion: 100%|█████████████████| 10/10 [00:10<00:00,  1.04s/it]

CLASS columns converted and NaNs filled


In [9]:
# ============================================================
# 4️) CLASS SUM CHECK (with progress)
# ============================================================

class_cols = [c for c in traffic.columns if c.upper().startswith('CLASS')]

if class_cols:
    print("\n🔄 Calculating class sums...")
    traffic['class_sum'] = 0
    for c in tqdm(class_cols, desc="Summing CLASS columns"):
        traffic['class_sum'] += traffic[c].fillna(0).astype(float)
    traffic['class_mismatch'] = (traffic['TOTAL'] - traffic['class_sum']).abs() > 0.01
    mismatch_count = traffic['class_mismatch'].sum()
    print(f"Class mismatch rows: {mismatch_count}")


🔄 Calculating class sums...


Summing CLASS columns: 100%|████████████████████| 10/10 [00:00<00:00, 61.72it/s]

Class mismatch rows: 0


In [11]:
# --- Drop redundant columns
traffic = traffic.drop(columns=['Month', 'Yr', 'FAC_G2'], errors='ignore')

In [ ]:
# ============================================================
# 5️) STRATIFIED SAMPLING (5%) — progress optional
# ============================================================

#traffic['month_year'] = traffic['DATE'].dt.to_period('M')

#print("\n Creating stratified 5% sample (safe for all pandas versions)...")
#sample = (
#    traffic.groupby(['FAC_B', 'month_year'], group_keys=False)
#           .progress_apply(lambda x: x.sample(frac=0.05, random_state=1) if len(x) > 1 else x)
#           .reset_index(drop=True)  # ensures no duplicate index issues
#)
#print(f"Stratified sample created. Sample shape: {sample.shape}")



In [13]:
# ============================================================
# 6️) SAVE FINAL DATASETS
# ============================================================

from tqdm import tqdm
import time  # just for simulating progress

# List of files to save with descriptions
save_tasks = [
    #('EDA_All_Recording_Traffic_Sample.csv', lambda: sample.to_csv('EDA_Ready_All_Recording_Traffic_Sample.csv', index=False), 'Sample dataset (5%)'),
    ('EDA_All_Recording_Traffic.parquet', lambda: traffic.to_parquet('EDA_Ready_All_Recording_Traffic.parquet', index=False), 'Full cleaned dataset'),
    ('EDA_All_Recording_Traffic.txt', lambda: traffic.to_csv('EDA_Ready_All_Recording_Traffic.txt', sep='\t', index=False), 'Full dataset tab-delimited')
]

print("\n Saving final datasets with progress:")

for filename, func, desc in tqdm(save_tasks, desc="Saving files", unit="file"):
    func()  # execute the save function
    # Optional: tiny sleep to visually show progress if fast
    time.sleep(0.1)

print("\n All steps completed successfully.")
print("➡ Saved files:")
#print("   - EDA_All_Recording_Traffic_Sample.csv  (for quick analysis)")
print("   - EDA_All_Recording_Traffic.parquet  (full cleaned dataset)")
print("   - EDA_All_Recording_Traffic.txt  (full dataset tab-delimited)")



 Saving final datasets with progress:


Saving files: 100%|█████████████████████████████| 2/2 [00:35<00:00, 17.95s/file]


 All steps completed successfully.
➡ Saved files:
   - EDA_All_Recording_Traffic.parquet  (full cleaned dataset)
   - EDA_All_Recording_Traffic.txt  (full dataset tab-delimited)


In [ ]:
#!pip install holidays

In [19]:
# Before running this script, perform exploratory data analysis to identify missing values
# and any necessary type conversions or data cleansing.
# The following code performs feature engineering to prepare the 'All Recording Traffic' dataset for Power BI:

import os
import pandas as pd
from tqdm import tqdm
from pandas.tseries.holiday import USFederalHolidayCalendar

tqdm.pandas(desc="Processing rows")

# 1. Load your traffic data from TXT
df = pd.read_csv("EDA_Ready_All_Recording_Traffic.txt", sep="\t")

# 2. Convert DATE and DATETIME to datetime
df["DATE"] = pd.to_datetime(df["DATE"])
df["DATETIME"] = pd.to_datetime(df["DATETIME"])

# 3. Extract Year and Month
df["Year"] = df["DATE"].dt.year
df["Month"] = df["DATE"].dt.month

# 4. Is_Weekend flag
df["Is_Weekend"] = df["Day_Name"].progress_apply(lambda d: 1 if d in ["Saturday","Sunday"] else 0)

# 5. Is_Holiday flag using pandas holiday calendar
cal = USFederalHolidayCalendar()
holidays = cal.holidays(start=df["DATE"].min(), end=df["DATE"].max())
df["Is_Holiday"] = df["DATE"].progress_apply(lambda d: 1 if d in holidays else 0)

# 6. Season
def get_season(m):
    if m in [12,1,2]:
        return "Winter"
    elif m in [3,4,5]:
        return "Spring"
    elif m in [6,7,8]:
        return "Summer"
    else:
        return "Fall"
df["Season"] = df["Month"].progress_apply(get_season)

# 7. Violation_Rate (vectorized)
df["Violation_Rate"] = (df["VIOLATION"] / df["TOTAL"]).fillna(0)

# 8. Week_of_Year
df["Week_of_Year"] = df["DATE"].dt.isocalendar().week

# 9. Hour and Time_of_Day
df["Hour"] = df["DATETIME"].dt.hour
def tod(h):
    if 6 <= h < 12:
        return "Morning"
    elif 12 <= h < 18:
        return "Afternoon"
    elif 18 <= h < 22:
        return "Evening"
    else:
        return "Night"
df["Time_of_Day"] = df["Hour"].progress_apply(tod)

# 10. Lagged and rolling features
df = df.sort_values(["FAC_B","DATETIME"])
df["Total_Lag1D"] = df.groupby("FAC_B")["TOTAL"].shift(24)
df["Total_Rolling_3D"] = (
    df.groupby("FAC_B")["TOTAL"]
      .transform(lambda x: x.rolling(window=72, min_periods=1).mean())
)

# 12. STRATIFIED SAMPLING (5%)
df["month_year"] = df["DATE"].dt.to_period("M")
print("\nCreating stratified 5% sample (safe for all pandas versions)...")
sample = (
    df.groupby(['FAC_B', 'month_year'], group_keys=False)
      .progress_apply(lambda x: x.sample(frac=0.05, random_state=1) if len(x) > 1 else x)
      .reset_index(drop=True)
)
print(f"Stratified sample created. Sample shape: {sample.shape}")


# Save full dataset and sample
output_file = "data/output/Traffic_PowerBI_Ready.csv"

# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(output_file), exist_ok=True)

df.to_csv(output_file, index=False)
print(f"\nFeature engineering complete! Saved full dataset to: {output_file}")

sample_file = "data/output/Traffic_PowerBI_Sample.csv"
sample.to_csv(sample_file, index=False)
print(f"Sample saved to: {sample_file}")

Processing rows: 100%|███████████| 5383378/5383378 [00:01<00:00, 3438012.81it/s]



Creating stratified 5% sample (safe for all pandas versions)...


Processing rows: 100%|█████████████████████| 1158/1158 [00:01<00:00, 618.94it/s]


Stratified sample created. Sample shape: (269176, 43)

Feature engineering complete! Saved full dataset to: data/output/Traffic_PowerBI_Ready.csv
Sample saved to: data/output/Traffic_PowerBI_Sample.csv


In [21]:
# (Optional) --- Monthly Aggregation Step ---

from tqdm import tqdm
import pandas as pd

monthly_output_file = "data/output/Traffic_PowerBI_Ready_Monthly.csv"
ready_file = "data/output/Traffic_PowerBI_Ready.csv"

# Read with progress bar based on file size
print("\nLoading PowerBI-ready file for monthly aggregation...")
df_ready = pd.read_csv(ready_file)
print(f"Loaded {df_ready.shape[0]} rows, {df_ready.shape[1]} columns.")

# Remove unnecessary columns (with tqdm for column removal progress)
remove_cols = [
    'DAY', 'DATE', 'TIME', 'Day_Name', 'TIME_parsed', 'DATETIME', 'class_sum', 'class_mismatch',
    'Year', 'Month', 'Is_Weekend', 'Is_Holiday', 'Violation_Rate', 'Week_of_Year',
    'Hour', 'Time_of_Day', 'Total_Lag1D', 'Total_Rolling_3D'
]
print("Dropping unnecessary columns...")
cols_to_drop = [col for col in tqdm(remove_cols, desc="Checking columns to drop") if col in df_ready.columns]
df_monthly = df_ready.drop(columns=cols_to_drop)

# tqdm for aggregation progress (simulate per group/row if desired)
#group_keys = ['month_year', 'FAC', 'LANE', 'FAC_B']
group_keys = ['month_year', 'FAC', 'FAC_B']
value_cols = [col for col in df_monthly.columns if col not in group_keys]

print("Aggregating monthly values (this may take some time)...")
df_monthly_agg = df_monthly.groupby(group_keys, group_keys=False)[value_cols].sum(numeric_only=True).reset_index()

# Check how many rows per FAC per month
print(df_monthly_agg.groupby(['month_year', 'FAC']).size().unique())

# Save and confirm
df_monthly_agg.to_csv(monthly_output_file, index=False)
print(f"\nMonthly aggregated data saved to: {monthly_output_file}, shape: {df_monthly_agg.shape}")



Loading PowerBI-ready file for monthly aggregation...
Loaded 5383378 rows, 43 columns.
Dropping unnecessary columns...


Checking columns to drop: 100%|██████████████| 18/18 [00:00<00:00, 67168.57it/s]


Aggregating monthly values (this may take some time)...
[1]

Monthly aggregated data saved to: data/output/Traffic_PowerBI_Ready_Monthly.csv, shape: (1158, 22)


In [25]:
# ------------------ NEW SECTION: Monthly Lag and Rolling Features ------------------

print("\nAdding monthly lag and rolling average features...")

# Ensure datetime and proper sorting
df_monthly_agg["month_year"] = pd.to_datetime(df_monthly_agg["month_year"])
df_monthly_agg = df_monthly_agg.sort_values(["FAC", "month_year"])

# Add lag features per facility
df_monthly_agg["TOTAL_LAG_1M"] = df_monthly_agg.groupby("FAC")["TOTAL"].shift(1)
df_monthly_agg["TOTAL_LAG_3M"] = df_monthly_agg.groupby("FAC")["TOTAL"].shift(3)

# Add rolling mean feature (6-month moving average)
df_monthly_agg["TOTAL_ROLL_6M"] = (
    df_monthly_agg.groupby("FAC")["TOTAL"]
      .transform(lambda x: x.rolling(window=6, min_periods=1).mean())
)

# Optionally fill NaNs from lags
df_monthly_agg[["TOTAL_LAG_1M", "TOTAL_LAG_3M"]] = df_monthly_agg[["TOTAL_LAG_1M", "TOTAL_LAG_3M"]].bfill()


# Save enhanced dataset
enhanced_output_file = "data/output/Traffic_PowerBI_Ready_Monthly_Enhanced.csv"
df_monthly_agg.to_csv(enhanced_output_file, index=False)
print(f"Enhanced monthly dataset with lag/rolling features saved to: {enhanced_output_file}")


Adding monthly lag and rolling average features...
✅ Enhanced monthly dataset with lag/rolling features saved to: data/output/Traffic_PowerBI_Ready_Monthly_Enhanced.csv


In [43]:
# For Clustering Analysis
import pandas as pd
import numpy as np
from tqdm import tqdm

tqdm.pandas()  # Enable progress bars

# -------------------------------
# Load your monthly aggregated dataset
# -------------------------------
df_monthly_agg = pd.read_csv("data/output/Traffic_PowerBI_Ready_Monthly.csv")

# -------------------------------
# Ensure datetime and proper sorting
# -------------------------------
df_monthly_agg["month_year"] = pd.to_datetime(df_monthly_agg["month_year"])
df_monthly_agg = df_monthly_agg.sort_values(["FAC", "month_year"])

# -------------------------------
# Add rolling mean feature (6-month moving average)
# -------------------------------
print("➡️  Adding 6-month rolling total...")
df_monthly_agg["TOTAL_ROLL_6M"] = (
    df_monthly_agg.groupby("FAC")["TOTAL"]
      .transform(lambda x: x.rolling(window=6, min_periods=1).mean())
)

# -------------------------------
# Add violation and payment ratios
# -------------------------------
print("➡️  Calculating violation and payment ratios...")
df_monthly_agg["VIOLATION_RATIO"] = df_monthly_agg["VIOLATION"] / df_monthly_agg["TOTAL"]
df_monthly_agg["CASH_RATIO"] = df_monthly_agg["CASH"] / df_monthly_agg["TOTAL"]
df_monthly_agg["EZPASS_RATIO"] = df_monthly_agg["EZPASS"] / df_monthly_agg["TOTAL"]
df_monthly_agg["AUTOS_RATIO"] = df_monthly_agg["Autos"] / df_monthly_agg["TOTAL"]
df_monthly_agg["SMALL_T_RATIO"] = df_monthly_agg["Small_T"] / df_monthly_agg["TOTAL"]
df_monthly_agg["LARGE_T_RATIO"] = df_monthly_agg["Large_T"] / df_monthly_agg["TOTAL"]
df_monthly_agg["BUSES_RATIO"] = df_monthly_agg["Buses"] / df_monthly_agg["TOTAL"]

# -------------------------------
# Add yearly growth indicators
# -------------------------------
print("➡️  Adding yearly growth indicators...")
df_monthly_agg["YEAR"] = df_monthly_agg["month_year"].dt.year

# Compute annual mean
df_monthly_agg["ANNUAL_TOTAL_MEAN"] = (
    df_monthly_agg.groupby(["FAC", "YEAR"], group_keys=False)["TOTAL"]
    .transform("mean")
)

# Compute year-over-year growth
df_monthly_agg["YEARLY_GROWTH"] = (
    df_monthly_agg.groupby("FAC", group_keys=False)["ANNUAL_TOTAL_MEAN"]
    .transform(lambda x: x.diff())
)

# Option 1: Fill NaN with 0
df_monthly_agg["YEARLY_GROWTH"] = df_monthly_agg["YEARLY_GROWTH"].fillna(0)

# -------------------------------
# Save enhanced dataset (keep all original columns + new features)
# -------------------------------
enhanced_output_file = "data/output/Traffic_Clustering_Ready_Monthly.csv"
df_monthly_agg.to_csv(enhanced_output_file, index=False)
print(f"✅ Enhanced dataset with new features saved to: {enhanced_output_file}")


➡️  Adding 6-month rolling total...
➡️  Calculating violation and payment ratios...
➡️  Adding yearly growth indicators...
✅ Enhanced dataset with new features saved to: data/output/Traffic_Clustering_Ready_Monthly.csv


In [19]:
# (Optional) --- Weekly Aggregation Step ---

from tqdm import tqdm
import pandas as pd

ready_file = "data/output/Traffic_PowerBI_Ready.csv"
weekly_output_file = "data/output/Traffic_PowerBI_Ready_Weekly.csv"

print("\nLoading PowerBI-ready file for weekly aggregation...")
df_ready = pd.read_csv(ready_file)
print(f"Loaded {df_ready.shape[0]} rows, {df_ready.shape[1]} columns.")

remove_cols = [
    'DAY', 'TIME', 'Day_Name', 'TIME_parsed', 'DATETIME', 'class_sum', 'class_mismatch',
    'Year', 'Month', 'Is_Weekend', 'Is_Holiday', 'Violation_Rate', 'Week_of_Year',
    'Hour', 'Time_of_Day', 'Total_Lag1D', 'Total_Rolling_3D'
]
print("Dropping unnecessary columns for weekly aggregation...")
cols_to_drop = [col for col in tqdm(remove_cols, desc="Checking columns to drop") if col in df_ready.columns]
df_weekly = df_ready.drop(columns=cols_to_drop)

# Create a consistent week label if not present
df_weekly['Week_of_Year'] = pd.to_datetime(df_weekly['DATE']).dt.isocalendar().week
df_weekly['Year'] = pd.to_datetime(df_weekly['DATE']).dt.year

# Optionally keep Is_Holiday, Is_Weekend, Day_Name, Season, etc. as "first"/"any" if useful for context
group_keys = ['Year', 'Week_of_Year', 'FAC', 'LANE', 'FAC_B']
value_cols = [col for col in df_weekly.columns if col not in group_keys]

print("Aggregating weekly values (may take time)...")
df_weekly_agg = df_weekly.groupby(group_keys, group_keys=False)[value_cols].sum(numeric_only=True).reset_index()

df_weekly_agg.to_csv(weekly_output_file, index=False)
print(f"\nWeekly aggregated data saved to: {weekly_output_file}, shape: {df_weekly_agg.shape}")



Loading PowerBI-ready file for weekly aggregation...
Loaded 5383378 rows, 43 columns.
Dropping unnecessary columns for weekly aggregation...


Checking columns to drop: 100%|██████████████| 17/17 [00:00<00:00, 64411.17it/s]


Aggregating weekly values (may take time)...

Weekly aggregated data saved to: data/output/Traffic_PowerBI_Ready_Weekly.csv, shape: (37484, 23)


In [5]:
# (Optional) --- Daily Aggregation Step ---

from tqdm import tqdm
import pandas as pd

ready_file = "data/output/Traffic_PowerBI_Ready.csv"
daily_output_file = "data/output/Traffic_PowerBI_Ready_Daily.csv"

print("\nLoading PowerBI-ready file for daily aggregation...")
df_ready = pd.read_csv(ready_file)
print(f"Loaded {df_ready.shape[0]} rows, {df_ready.shape[1]} columns.")

remove_cols = [
    'DAY', 'TIME', 'TIME_parsed', 'DATETIME', 'class_sum', 'class_mismatch',
    'Year', 'Month', 'Season', 'Violation_Rate', 'Week_of_Year',
    'Hour', 'Time_of_Day', 'Total_Lag1D', 'Total_Rolling_3D'
]
print("Dropping unnecessary columns for daily aggregation...")
cols_to_drop = [col for col in tqdm(remove_cols, desc="Checking columns to drop") if col in df_ready.columns]
df_daily = df_ready.drop(columns=cols_to_drop)

# For aggregation, keep DATE, FAC, LANE, FAC_B, Is_Holiday, Is_Weekend, Day_Name
group_keys = ['DATE', 'FAC', 'LANE', 'FAC_B', 'Is_Holiday', 'Is_Weekend', 'Day_Name']
value_cols = [col for col in df_daily.columns if col not in group_keys]

print("Aggregating daily values (may take time)...")
df_daily_agg = df_daily.groupby(group_keys, group_keys=False)[value_cols].sum(numeric_only=True).reset_index()

df_daily_agg.to_csv(daily_output_file, index=False)
print(f"\nDaily aggregated data saved to: {daily_output_file}, shape: {df_daily_agg.shape}")



Loading PowerBI-ready file for daily aggregation...
Loaded 5383378 rows, 43 columns.
Dropping unnecessary columns for daily aggregation...


Checking columns to drop: 100%|██████████████| 15/15 [00:00<00:00, 48358.62it/s]


Aggregating daily values (may take time)...

Daily aggregated data saved to: data/output/Traffic_PowerBI_Ready_Daily.csv, shape: (250044, 25)


In [65]:
#   This script prepares a balanced dataset to address the 
#   AutoML classification alert: "Imbalanced classes were detected in your inputs."


# Script: Balanced dataset for AutoML without altering DATE

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.combine import SMOTEENN
from collections import Counter
from tqdm import tqdm

tqdm.pandas()

# ===============================
# Load dataset
# ===============================
df = pd.read_csv("data/output/Traffic_Classification_Ready_Daily.csv")
print("✅ Original dataset loaded.")
print("Shape:", df.shape)
print("Class distribution:\n", df['VIOLATION_FLAG'].value_counts())

# ===============================
# Encode all non-numeric columns except DATE
# ===============================
label_encoders = {}
for col in df.select_dtypes(include=['object']).columns:
    if col != 'DATE':  # exclude DATE from encoding
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        label_encoders[col] = le

print("\n✅ Non-numeric columns converted (excluding DATE).")

# ===============================
# Split features & target
# ===============================
# Exclude DATE from SMOTE features
features_to_balance = df.drop(columns=['VIOLATION_FLAG', 'DATE'])
target = df['VIOLATION_FLAG']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    features_to_balance, target,
    test_size=0.2,
    stratify=target,
    random_state=42
)

print("\nTraining set class distribution (before balancing):")
print(Counter(y_train))

# ===============================
# Apply SMOTE+ENN only on numeric features
# ===============================
class TQDSMOTEENN(SMOTEENN):
    def fit_resample(self, X, y):
        for _ in tqdm(range(1), desc="SMOTE+ENN in progress", ncols=80):
            return super().fit_resample(X, y)

sme = TQDSMOTEENN(random_state=42)
X_train_bal, y_train_bal = sme.fit_resample(X_train, y_train)

print("\n✅ Balancing complete!")
print("Training set class distribution (after balancing):")
print(Counter(y_train_bal))

# ===============================
# Add back the DATE column to training & test sets
# ===============================
# For training set, match original indices where possible
train_indices = X_train.index
X_train_bal_df = pd.DataFrame(X_train_bal, columns=X_train.columns)

# For simplicity, keep DATE as NaN for synthetic samples, or fill with a placeholder
X_train_bal_df['DATE'] = pd.NA  

# Test set keeps original DATE
X_test_df = X_test.copy()
X_test_df['DATE'] = df.loc[X_test.index, 'DATE'].values

# Combine features + target
balanced_train = pd.concat([X_train_bal_df,
                            pd.Series(y_train_bal, name='VIOLATION_FLAG')], axis=1)
balanced_test = pd.concat([X_test_df.reset_index(drop=True),
                           y_test.reset_index(drop=True)], axis=1)

# ===============================
# Save balanced datasets
# ===============================
balanced_train.to_csv("data/output/Traffic_Classification_Balanced.csv", index=False)
balanced_test.to_csv("data/output/Traffic_Classification_Test.csv", index=False)

print("\n✅ Files saved successfully.")



✅ Original dataset loaded.
Shape: (250044, 26)
Class distribution:
 VIOLATION_FLAG
1    244744
0      5300
Name: count, dtype: int64

✅ Non-numeric columns converted (excluding DATE).

Training set class distribution (before balancing):
Counter({1: 195795, 0: 4240})


SMOTE+ENN in progress:   0%|                              | 0/1 [29:08<?, ?it/s]



✅ Balancing complete!
Training set class distribution (after balancing):
Counter({0: 195738, 1: 194202})

✅ Files saved successfully.


In [24]:
# Note: This script calls / executes another Python script/notebook 
# ('data_ingestion_facility_speeds.ipynb' or its converted .py version) 
# to ensure the required data file 'Facility_Mobility_Speeds_Clean.csv' is generated beforehand.

#import nbformat
#from nbconvert.preprocessors import ExecutePreprocessor

#def run_notebook(path):
#    with open(path) as f:
#        nb = nbformat.read(f, as_version=4)
#    ep = ExecutePreprocessor(timeout=600, kernel_name='python3')
#    ep.preprocess(nb, {'metadata': {'path': './'}})  # Change path if needed

#run_notebook('data_ingestion_facility_speeds.ipynb')


# Facility Mobility Speeds file cleaning. Removing extra columns

import pandas as pd
from tqdm import tqdm

# 1. Load the TXT file (tab-separated)
df = pd.read_csv("Facility Mobility Speeds.txt", sep="\t")

# 2. Drop the static Facility_Order column
df = df.drop(columns=["Facility_Order"])

# 3. Format Month_Year with a progress bar
tqdm.pandas(desc="Formatting Month_Year")
df["Month_Year"] = df["Month_Year"].progress_apply(lambda x: pd.to_datetime(x).strftime("%Y-%m"))

# 4. Save the cleaned data for use in Power BI or further analysis
output_file = "Facility_Mobility_Speeds_Clean.csv"
df.to_csv(output_file, index=False)

# 5. Display a summary message
print(f"\nFinal cleaned data ({df.shape[0]} records, {df.shape[1]} columns) saved to: {output_file}")

# Facility mobility speeds - Feature Engineering for Predictive Analysis (if required)

import pandas as pd
from tqdm import tqdm

# Load cleaned data
df = pd.read_csv("Facility_Mobility_Speeds_Clean.csv")

# Feature engineering with progress bar
tqdm.pandas(desc="Feature Engineering")

# 1. Extract year and month from Month_Year
df["Month_Year"] = pd.to_datetime(df["Month_Year"], format="%Y-%m")
df["Year"] = df["Month_Year"].dt.year
df["Month"] = df["Month_Year"].dt.month

# 2. One-hot encode Direction
df = pd.get_dummies(df, columns=["Direction"], prefix="Dir")

# 3. (Optional) Create lag features for Avg_Speed (previous month per facility)
df = df.sort_values(["Facility", "Month_Year"])
df["Avg_Speed_Lag1"] = df.groupby("Facility")["Avg_Speed"].shift(1)

# 4. (Optional) Standardize continuous features
#for col in ["Freeflow", "Avg_Speed", "Delta"]:
#    mean = df[col].mean()
#    std = df[col].std()
#    df[f"{col}_z"] = (df[col] - mean) / std

# 5. Save engineered features
output_file = "Facility_Mobility_Speeds_Features.csv"
df.to_csv(output_file, index=False)

print(f"\nFeature engineered data ({df.shape[0]} records, {df.shape[1]} columns) saved to: {output_file}")

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [26]:
"""
Data Pipeline: Traffic and Mobility Data Merge and Aggregation

PURPOSE:
Aggregates hourly traffic data ('Traffic_PowerBI_Ready.csv') to a monthly level,
harmonizes facility names with mobility speed data ('Facility_Mobility_Speeds_Clean.csv'), 
performs an outer merge, and saves two distinct output files.

INPUTS:
1. 'Traffic_PowerBI_Ready.csv': Hourly traffic counts, requires aggregation by Month/Facility.
2. 'Facility_Mobility_Speeds_Clean.csv': Already aggregated monthly speed data using short facility codes.
    ** Note: To generate this file, run data_ingestion_facility_speeds.ipynb beforehand.

OUTPUTS:
1. 'Merged_Traffic_Mobility_Data_FULL.csv': 
    - Contains all combinations of Facility and Month from both input files (Outer Join).
    - Missing metric values (where data existed in one file but not the other) are filled with 0.
2. 'Merged_Traffic_Mobility_Data_FILTERED.csv':
    - Filtered version of the FULL dataset.
    - Only includes rows where both 'Aggregated_Traffic_Volume' (from Traffic) AND 'Freeflow' 
      (the primary speed metric) are non-zero. Use this file for direct comparison analysis.

ASSUMPTIONS & NOTES:
- The 'Traffic_PowerBI_Ready.csv' file uses the column 'TOTAL' for traffic volume.
- The traffic data contains historical records (e.g., 2013-2025), while the speed data 
  is currently limited (e.g., 2024-2025). The FULL merge preserves all historical traffic 
  data alongside available speed data.
- Facility names are mapped using the SHORT_TO_LONG_MAPPING defined below.
"""

import pandas as pd
import numpy as np

# --- 1. Define Mapping ---
# Mapping from Facility short code (in Facility_Mobility_Speeds_Clean) 
# to full name (in Traffic_PowerBI_Ready)
SHORT_TO_LONG_MAPPING = {
    "BB": "Bayonne Bridge",
    "GB": "Goethals Bridge",
    # The GWB short code maps to 'GWB Lower' for consistency when merging with the 
    # granular traffic data which may contain 'GWB Lower' and 'GWB PIP'.
    "GWB": "GWB Lower", 
    "HT": "Holland Tunnel",
    "LT": "Lincoln Tunnel",
    "OBX": "Outerbridge Crossing"
}

# --- 2. Load Data ---

# Load Traffic_PowerBI_Ready.csv (hourly data with FAC_B and 'TOTAL' column)
df_traffic = pd.read_csv('data/output/Traffic_PowerBI_Ready.csv') 
print("Traffic_PowerBI_Ready.csv loaded successfully.")

# DEBUGGING STEP: Removed after identifying the issue (TOTAL vs Total)

# Load Facility_Mobility_Speeds_Clean.csv (aggregated speed data with 'Facility' short code)
df_speeds = pd.read_csv("Facility_Mobility_Speeds_Clean.csv")
print("Facility_Mobility_Speeds_Clean.csv loaded successfully.")


# --- 3. Process Traffic_PowerBI_Ready (Aggregation) ---
print("\nProcessing Traffic Data...")

# Convert the time column ('month_year') to a monthly period for aggregation
df_traffic['Month_Year'] = pd.to_datetime(df_traffic['month_year']).dt.to_period('M')

# Aggregate the hourly data by FAC_B and Month_Year, summing the CORRECT 'TOTAL' column.
df_traffic_agg = df_traffic.groupby(['FAC_B', 'Month_Year']).agg(
    Aggregated_Traffic_Volume=('TOTAL', 'sum')  # <-- CORRECTED: Changed 'Total' to 'TOTAL'
).reset_index()

# Rename the facility column to the common merging key
df_traffic_agg.rename(columns={'FAC_B': 'Facility_Name'}, inplace=True)
print(f"Traffic data aggregated and has {len(df_traffic_agg)} monthly records.")


# --- 4. Process Facility_Mobility_Speeds_Clean (Mapping) ---
print("\nProcessing Mobility Speed Data...")

# Convert the time column ('Month_Year') to a monthly period for merging
df_speeds['Month_Year'] = pd.to_datetime(df_speeds['Month_Year']).dt.to_period('M')

# Map the short Facility codes (BB, GB) to the full names 
df_speeds['Facility_Name'] = df_speeds['Facility'].replace(SHORT_TO_LONG_MAPPING)

# Drop the old short code column
df_speeds.drop(columns=['Facility'], inplace=True)
print("Facility short codes replaced with full names.")


# --- 5. Merge Data and Fill NaN ---
print("\nMerging DataFrames...")

# Perform an 'outer' merge on the common keys ['Facility_Name', 'Month_Year']
merged_df = pd.merge(
    df_traffic_agg, 
    df_speeds, 
    on=['Facility_Name', 'Month_Year'], 
    how='outer'
)

# Identify the columns that need to be zero-filled (the numeric metric columns)
key_columns = ['Facility_Name', 'Month_Year']
metric_columns = [col for col in merged_df.columns if col not in key_columns]

# Fill non-matching metric values (NaNs from the outer join) with 0
merged_df[metric_columns] = merged_df[metric_columns].fillna(0)

# Convert Month_Year back to a standard string format for final output
merged_df['Month_Year'] = merged_df['Month_Year'].astype(str)

print("\n--- Final Merged DataFrame Head ---")
print(merged_df.head())
print(f"\nShape of the final merged DataFrame: {merged_df.shape}")

# --- 6. Save the Merged Data ---
merged_df.to_csv('data/output/Merged_Traffic_Mobility_Data_Full.csv', index=False)
print("\nData successfully saved to Merged_Traffic_Mobility_Data_Full.csv")


# --- 6. Filter for Non-Zero Metrics (Create the FILTERED Dataframe) ---
# Filter to keep only rows where the main traffic metric AND 
# the primary speed metric (Freeflow) are non-zero.
# We create a new DataFrame 'filtered_df' for this specific use case.

filtered_df = merged_df.copy()

if 'Freeflow' in filtered_df.columns:
    print("\nFiltering data to keep only rows where Aggregated_Traffic_Volume and Freeflow are non-zero...")
    filtered_df = filtered_df[
        (filtered_df['Aggregated_Traffic_Volume'] != 0) & 
        (filtered_df['Freeflow'] != 0)
    ].copy()
else:
    print("\nWarning: 'Freeflow' column not found for non-zero filtering. Skipping non-zero filter for filtered file.")

# Convert Month_Year back to a standard string format for final output (FILTERED data)
filtered_df['Month_Year'] = filtered_df['Month_Year'].astype(str)

print("\n--- Saving Filtered Merged DataFrame ---")
print(filtered_df.head())
print(f"Shape of the final merged and filtered DataFrame: {filtered_df.shape}")

# --- 7. Save the Filtered Data ---
filtered_df.to_csv('data/output/Merged_Traffic_Mobility_Data_FILTERED.csv', index=False)
print("Data successfully saved to Merged_Traffic_Mobility_Data_FILTERED.csv")


Traffic_PowerBI_Ready.csv loaded successfully.
Facility_Mobility_Speeds_Clean.csv loaded successfully.

Processing Traffic Data...
Traffic data aggregated and has 1158 monthly records.

Processing Mobility Speed Data...
Facility short codes replaced with full names.

Merging DataFrames...

--- Final Merged DataFrame Head ---
  Facility_Name Month_Year  Aggregated_Traffic_Volume  Freeflow  Avg_Speed  \
0       Bayonne    2013-01                   279932.0       0.0        0.0   
1       Bayonne    2013-02                   249747.0       0.0        0.0   
2       Bayonne    2013-03                   283081.0       0.0        0.0   
3       Bayonne    2013-04                   288071.0       0.0        0.0   
4       Bayonne    2013-05                   302573.0       0.0        0.0   

  Direction  Delta  
0         0    0.0  
1         0    0.0  
2         0    0.0  
3         0    0.0  
4         0    0.0  

Shape of the final merged DataFrame: (1334, 7)

Data successfully saved to Me

In [ ]:
# Objective: Generate a dataset with traffic volume categories (High / Medium / Low)
# for each facility, so it can be used to train a classification model in AutoML
# to predict future traffic volume categories based on daily features.

# To achieve this, we:
# 1. Aggregate lane-level daily traffic data to facility-level totals.
# 2. Apply clustering on total traffic for each facility to assign High / Medium / Low labels.
# 3. Visualize clusters to understand traffic patterns and support future predictive modeling.

import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

# ---------- LOAD DATA ----------
df = pd.read_csv("data/output/Traffic_Classification_Ready_Daily.csv", parse_dates=['DATE'])

# ------------------- AGGREGATE DAILY DATA PER FACILITY -------------------
agg_cols = ['TOTAL', 'Autos', 'Small_T', 'Large_T', 'Buses', 'VIOLATION']
daily_fac_df = df.groupby(['DATE','FAC']).agg({col:'sum' for col in agg_cols}).reset_index()

print("Unique facilities:", sorted(daily_fac_df['FAC'].unique()))
print(daily_fac_df['FAC'].value_counts())

# Keep relevant features for clustering
cluster_features = ['TOTAL']  # you can add other numeric columns if needed

# ------------------- CLUSTERING PER FACILITY -------------------
volume_levels = []
facilities = sorted(daily_fac_df['FAC'].unique())  # only existing FAC values

for fac in facilities:
    fac_df = daily_fac_df[daily_fac_df['FAC'] == fac].copy()
    
    # Using 3 clusters for High, Medium, Low
    kmeans = KMeans(n_clusters=3, random_state=42)
    fac_df['Cluster'] = kmeans.fit_predict(fac_df[cluster_features])
    
    # Map clusters to High/Medium/Low based on mean TOTAL per cluster
    cluster_means = fac_df.groupby('Cluster')['TOTAL'].mean().sort_values(ascending=False)
    mapping = {cluster: level for cluster, level in zip(cluster_means.index, ['High','Medium','Low'])}
    fac_df['Volume_Level'] = fac_df['Cluster'].map(mapping)
    
    volume_levels.append(fac_df)

# Combine all facilities
final_df = pd.concat(volume_levels).sort_values(['FAC','DATE'])

# ------------------- VISUALIZE -------------------
# Separate figure per facility to avoid clutter
for fac in facilities:
    fac_df = final_df[final_df['FAC'] == fac]
    plt.figure(figsize=(10,4))
    sns.scatterplot(
        data=fac_df,
        x='DATE',
        y='TOTAL',
        hue='Volume_Level',
        palette='viridis'
    )
    plt.title(f'FAC {fac} Traffic Clusters Over Time')
    plt.xlabel('Date')
    plt.ylabel('Total Traffic')
    plt.xticks(rotation=45)
    plt.legend(title='Volume Level')
    plt.tight_layout()
    plt.show()


# ---------- SAVE DATASETS ----------
# 1. For visualization (keeps internal Cluster info)
final_df.to_csv("data/output/Traffic_Facility_Clustered_With_Cluster.csv", index=False)

# 2. For AutoML (drops Cluster to avoid target leakage)
automl_df = final_df.drop(columns=['Cluster'])
automl_df.to_csv("data/output/Traffic_Facility_Clustered_For_Classification.csv", index=False)
